https://adventofcode.com/2015/day/24

In [1]:
input = r'''1
2
3
7
11
13
17
19
23
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97
101
103
107
109
113'''

In [2]:
test_input = r'''1
2
3
4
5
7
8
9
10
11'''

In [3]:
from itertools import *
from more_itertools import *
import numpy as np

In [4]:
def gen_best_legroom(weights, target_weight):
  for n in count():
    flag = False
    for center in combinations(weights, n):
      if sum(center) == target_weight:
        yield center
        flag = True
    if flag:
      return

def gen_valid_centers(weights, target_weight):
  for center in gen_best_legroom(weights, target_weight):
    for arr1, arr2 in set_partitions(set(weights)-set(center), 2):
      if sum(arr1) == target_weight:
        yield center
        break

def best_qe(weights, target_weight):
  best_qe = np.inf
  for center in gen_valid_centers(weights, target_weight):
    best_qe = min(np.prod(center), best_qe)
  return best_qe

weights = [int(line) for line in input.split('\n')]
total_weight = sum(weights)
target_weight = total_weight // 3
assert target_weight * 3 == total_weight
best_qe(weights, target_weight)

11846773891

In [5]:
weights = [int(line) for line in test_input.split('\n')]
total_weight = sum(weights)
target_weight = total_weight // 3
assert target_weight * 3 == total_weight
best_qe(weights, target_weight)

99

https://adventofcode.com/2015/day/24#part2

In [6]:
def gen_best_legroom(weights, target_weight):
  for n in count():
    flag = False
    for center in combinations(weights, n):
      if sum(center) == target_weight:
        yield center
        flag = True
    if flag:
      return

def gen_valid_centers(weights, target_weight):
  for center in gen_best_legroom(weights, target_weight):
    flag = False
    for arr1, arr23 in set_partitions(set(weights)-set(center), 2):
      if sum(arr1) == target_weight:
        for arr2, arr3 in set_partitions(arr23, 2):
          if sum(arr2) == target_weight:
            yield center
            flag=True
            break
      if flag:
        break

def best_qe(weights, target_weight):
  best_qe = np.inf
  for center in gen_valid_centers(weights, target_weight):
    best_qe = min(np.prod(center), best_qe)
  return best_qe

weights = [int(line) for line in input.split('\n')]
total_weight = sum(weights)
target_weight = total_weight // 4
assert target_weight * 4 == total_weight
best_qe(weights, target_weight)

80393059

In [7]:
# Don't do it this way, friends.

def assign(weights, target_weight, center=[], left=[], right=[], best_len_center=np.inf, best_qe=np.inf, debug=False):
  if any(sum(group) > target_weight for group in (center, left, right)):
    return None
  if len(center) > best_len_center:
    if debug:
      print('bailed early due to suboptimal leg room')
    return None
  if len(center) == best_len_center:
    if np.prod(center) > best_qe:
      if debug:
        print('bailed early due to suboptimal quantum entanglement')
      return None
    if sum(center) < target_weight:
      if debug:
        print('bailed early due to suboptimal legroom (type 2)')
      return None
  if not weights: # weights is empty
    return center, left, right, len(center), np.prod(center)
  one_weight, other_weights = weights[0], weights[1:]

  res1 = assign(other_weights, target_weight, center=[one_weight]+center, left=left, right=right, best_len_center=best_len_center, best_qe=best_qe)
  if res1 is not None:
    c1, l1, r1, lc1, pc1 = res1
    if lc1 < best_len_center:
      best_len_center = lc1
      best_qe = pc1
    elif lc1 == best_len_center:
      if pc1 < best_qe:
        best_qe = pc1 
  res2 = assign(other_weights, target_weight, center=center, left=[one_weight]+left, right=right, best_len_center=best_len_center, best_qe=best_qe)
  if res2 is not None:
    c2, l2, r2, lc2, pc2 = res2
    if lc2 < best_len_center:
      best_len_center = lc2
      best_qe = pc2
    elif lc2 == best_len_center:
      if pc2 < best_qe:
        best_qe = pc2
  res3 = assign(other_weights, target_weight, center=center, left=left, right=[one_weight]+right, best_len_center=best_len_center, best_qe=best_qe)
  if res3 is not None:
    c3, l3, r3, lc3, pc3 = res3
    if lc3 < best_len_center:
      best_len_center = lc3
      best_qe = pc3
    elif lc3 == best_len_center:
      if pc3 < best_qe:
        best_qe = pc3
  
  sub_results = [res1, res2, res3]
  sub_results = [x for x in sub_results if x is not None]

  if not sub_results:
    return None
  minimal_len_center = min(x[3] for x in sub_results)
  sub_results = [x for x in sub_results if x[3] == minimal_len_center]
  if not sub_results:
    return None
  minimal_prod_center = min(x[4] for x in sub_results)
  sub_results = [x for x in sub_results if x[4] == minimal_prod_center]
  if not sub_results:
    return None
  
  print('current best', sub_results[0][3], sub_results[0][4])
  return sub_results[0]

def main(input):
  weights = [int(line) for line in input.split('\n')]
  total_weight = sum(weights)
  target_weight = total_weight // 3
  assert target_weight * 3 == total_weight
  weights = weights[::-1]

  center, left, right, lc, pc = assign(weights, target_weight)
  return pc

In [ ]:
main(input)